In [ ]:
import os
import random
import pickle
import numpy as np
random.seed(42)
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


import time
import pandas as pd

import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.models import load_model
import gc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Master - A/june_muchomor_sromotnikowy_12500.pkl", "rb") as f:
    X_muchomor = pickle.load(f)
with open("/content/drive/MyDrive/Master - A/june_kania_12500.pkl", "rb") as f:
    X_kania = pickle.load(f)

labels_muchomory = list(np.zeros(len(X_muchomor)))
labels_kania = list(np.ones(len((X_kania))))

In [ ]:
from sklearn.utils import shuffle

X = np.array(X_kania + X_muchomor)
y = np.array(labels_kania + labels_muchomory)
X = X.reshape(-1, 224, 224, 3).astype('float32')
X = X.reshape(X.shape[0], -1)

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# pca = PCA(n_components=2000)
# X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

del X, y, X_muchomor, labels_muchomory, X_kania, labels_kania



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid, cross_val_score

model_classes = [
    ('KNN', KNeighborsClassifier(), {
        'n_neighbors': list(range(3, 21)),
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    }),

    ('DecisionTree', DecisionTreeClassifier(), {
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 5],
        'criterion': ['gini', 'entropy'],
        'max_features': [None, 'sqrt', 'log2']
    }),

    ('RandomForest', RandomForestClassifier(), {
        'n_estimators': [10, 25, 50, 100, 150, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }),

    ('Logistic regression', LogisticRegression(), {
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'C': [0.01, 0.1, 0.5, 1, 10, 100],
        'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg'],
        'max_iter': [50, 100, 150, 200, 250, 300, 250, 400, 450, 500]
    })
    ,

    ('AdaBoost', AdaBoostClassifier(), {
        'n_estimators': [50, 75, 100, 125, 150, 175, 200],
        'learning_rate': [0.01, 0.05, 0.1, 1.0]
    }),

    ('NaiveBayes', GaussianNB(), {
    })
]

def sim_grid(X_train, X_test, y_train, y_test, param=None):
    if param==None:
        param = ""

    results = []
    best = []

    for name, model, param_grid in model_classes:
        print(f"{name} - tuning...")

        grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        start_time = time.time()

        grid.fit(X_train, y_train)
        end_time = time.time()
        elapsed_time = end_time - start_time
        best_model = grid.best_estimator_
        joblib.dump(best_model, f"models/{param}_{name}_best_model.pkl")
        preds = best_model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_results.txt", "w") as f:
          f.write(f"Model: {name}\n")
          f.write(f"Training time: {elapsed_time:.2f} sec\n")
          f.write(f"Accuracy: {acc:.4f}\n\n")
          f.write("Classification Report:\n")

        #print(f"{name} - Best Params: {grid.best_params_}")
        #print(f"{name} - Test Accuracy: {acc:.4f}")
        #print(f"Training time: {elapsed_time:.2f} seconds")



        #print(classification_report(y_test, preds))

        conf_matrix = confusion_matrix(y_test, preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["E", "P"], yticklabels=["E", "P"])
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_matrix.png")
        plt.close()

        fpr, tpr, _ = roc_curve(y_test, preds)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate', fontsize=18)
        plt.ylabel('True Positive Rate', fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.legend(loc="lower right", fontsize=14)
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_roc_curve.png")
        plt.close()


        results.append((name, acc))
        best.append((name, grid.best_params_))

        results_df = pd.DataFrame(grid.cv_results_)
        results_df = pd.DataFrame(grid.cv_results_)
        param_cols = [col for col in results_df.columns if col.startswith('param_')]
        results_df = results_df[param_cols + ['mean_test_score', 'mean_fit_time', 'rank_test_score']]
        results_df.to_csv(f"/content/drive/MyDrive/Master - A/2_fig{param}_{name}.csv")

    for name, acc in results:
        print(f"{name}: {acc:.4f}")

    return best

In [ ]:
sim_grid(X_train[:5000], X_test, y_train[:5000], y_test, 5000)

KNN - tuning...


KeyboardInterrupt: 

In [ ]:
import joblib
import csv

In [ ]:
import time
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import csv

model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    ('DecisionTree_D', DecisionTreeClassifier()),
    ('RandomForest_D', RandomForestClassifier()),
    ('LogisticRegression_D', LogisticRegression())
]

param_grid = {
    'KNN_D': [{'n_neighbors': k} for k in [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]],
    'DecisionTree_D': [{'max_depth': d} for d in [5, 10, 15, 20, 25, 30, None]],
    'RandomForest_D': [{'n_estimators': n, 'max_depth': d, 'max_features': 'sqrt'} for n in [50, 100, 1000] for d in [10, 20]],
}

def classical_sim(X_train, X_test, y_train, y_test, param=120):
    results = []

    for name, model in model_classes:
        print(f"\n{name} - training...")

        for param_set in param_grid.get(name, []):
            print(f"Testing with parameters: {param_set}")

            start_time = time.time()

            model.set_params(**param_set)

            model.fit(X_train, y_train)
            end_time = time.time()
            training_time = end_time - start_time

            preds = model.predict(X_test)
            accuracy = accuracy_score(y_test, preds)
            report = classification_report(y_test, preds)

            with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_params_{param_set}_results.txt", "w") as f:
                f.write(f"Model: {name}\n")
                f.write(f"Parameters: {param_set}\n")
                f.write(f"Training time: {training_time:.2f} sec\n")
                f.write(f"Accuracy: {accuracy:.4f}\n\n")
                f.write("Classification Report:\n")
                f.write(report)

            conf_matrix = confusion_matrix(y_test, preds)
            plt.figure(figsize=(8, 6))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
            plt.title(f"{name} - Confusion Matrix")
            plt.xlabel("Predicted")
            plt.ylabel("Actual")
            plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_params_{param_set}_matrix.png")
            plt.close()

            fpr, tpr, _ = roc_curve(y_test, preds)
            roc_auc = auc(fpr, tpr)
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlabel('False Positive Rate', fontsize=18)
            plt.ylabel('True Positive Rate', fontsize=18)
            plt.xticks(fontsize=18)
            plt.yticks(fontsize=18)
            plt.legend(loc="lower right", fontsize=14)
            plt.tight_layout()
            plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_params_{param_set}_roc_curve.png")
            plt.close()

            results.append((name, param_set, accuracy))


    with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_final_results.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Parameters", "Accuracy"])
        for name, param_set, acc in results:
            writer.writerow([name, str(param_set), f"{acc:.4f}"])

    print("\nFinal Results:")
    for name, param_set, acc in results:
        print(f"{name} with parameters {param_set}: {acc:.4f}")



In [ ]:
classical_sim(X_train[:12000], X_test, y_train[:12000], y_test, param = "june_12")



KNN_D - training...
Testing with parameters: {'n_neighbors': 11}
Testing with parameters: {'n_neighbors': 12}
Testing with parameters: {'n_neighbors': 13}
Testing with parameters: {'n_neighbors': 14}
Testing with parameters: {'n_neighbors': 15}
Testing with parameters: {'n_neighbors': 16}
Testing with parameters: {'n_neighbors': 17}
Testing with parameters: {'n_neighbors': 18}
Testing with parameters: {'n_neighbors': 19}
Testing with parameters: {'n_neighbors': 20}
Testing with parameters: {'n_neighbors': 21}
Testing with parameters: {'n_neighbors': 22}
Testing with parameters: {'n_neighbors': 23}
Testing with parameters: {'n_neighbors': 24}
Testing with parameters: {'n_neighbors': 25}

DecisionTree_D - training...
Testing with parameters: {'max_depth': 5}
Testing with parameters: {'max_depth': 10}
Testing with parameters: {'max_depth': 15}
Testing with parameters: {'max_depth': 20}
Testing with parameters: {'max_depth': 25}
Testing with parameters: {'max_depth': 30}
Testing with para

In [ ]:
classical_sim(X_train[:16000], X_test, y_train[:16000], y_test, param = "june_16")



KNN_D - training...
Testing with parameters: {'n_neighbors': 11}
Testing with parameters: {'n_neighbors': 12}
Testing with parameters: {'n_neighbors': 13}
Testing with parameters: {'n_neighbors': 14}
Testing with parameters: {'n_neighbors': 15}
Testing with parameters: {'n_neighbors': 16}
Testing with parameters: {'n_neighbors': 17}
Testing with parameters: {'n_neighbors': 18}
Testing with parameters: {'n_neighbors': 19}
Testing with parameters: {'n_neighbors': 20}
Testing with parameters: {'n_neighbors': 21}
Testing with parameters: {'n_neighbors': 22}
Testing with parameters: {'n_neighbors': 23}
Testing with parameters: {'n_neighbors': 24}
Testing with parameters: {'n_neighbors': 25}

DecisionTree_D - training...
Testing with parameters: {'max_depth': 5}
Testing with parameters: {'max_depth': 10}


KeyboardInterrupt: 

In [ ]:
classical_sim(X_train, X_test, y_train, y_test, param = "june_20")


In [ ]:
model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    ('DecisionTree_D', DecisionTreeClassifier()),
    ('RandomForest_D', RandomForestClassifier()),
    #('AdaBoost_D', AdaBoostClassifier()),
    ('LogisticRegression_D', LogisticRegression()),
]

def classical_sim(X_train, X_test, y_train, y_test, param=120):
    results = []

    for name, model in model_classes:
        print(f"\n{name} - training...")
        start_time = time.time()

        model.fit(X_train, y_train)
        end_time = time.time()
        training_time = end_time - start_time

        preds = model.predict(X_test)
        acc_preview = accuracy_score(y_test, preds)

        y_prob = model.predict(X_test)
        y_pred = (y_prob > 0.5).astype("int32")
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_results.txt", "w") as f:
            f.write(f"Model: {name}\n")
            f.write(f"Training time: {training_time:.2f} sec\n")
            f.write(f"Accuracy: {accuracy:.4f}\n\n")
            f.write("Classification Report:\n")
            f.write(report)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_matrix.png")
        plt.close()

        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate', fontsize=18)
        plt.ylabel('True Positive Rate', fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.legend(loc="lower right", fontsize=14)
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}_roc_curve.png")
        plt.close()

        results.append((name, accuracy))

        del model, preds
        gc.collect()

    with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_final_results.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Accuracy"])
        for name, acc in results:
            writer.writerow([name, f"{acc:.4f}"])

    print("\nFinal Results:")
    for name, acc in results:
        print(f"{name}: {acc:.4f}")




In [ ]:
print(y_train.shape)
print(y_test.shape)

(19986,)
(4997,)


# 6000


In [ ]:
import csv
classical_sim(X_train[:12000], X_test, y_train[:12000], y_test, 112000)


KNN_D - training...

DecisionTree_D - training...

RandomForest_D - training...

AdaBoost_D - training...


KeyboardInterrupt: 

#8000

In [ ]:
classical_sim(X_train[:16000], X_test, y_train[:16000], y_test, 16000)



KNN_D - training...

DecisionTree_D - training...

RandomForest_D - training...

LogisticRegression_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Final Results:
KNN_D: 0.5549
DecisionTree_D: 0.6166
RandomForest_D: 0.7400
LogisticRegression_D: 0.6956


### 10000

In [ ]:
classical_sim(X_train, X_test, y_train, y_test, 20000)



KNN_D - training...

DecisionTree_D - training...

RandomForest_D - training...

LogisticRegression_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Final Results:
KNN_D: 0.5523
DecisionTree_D: 0.6302
RandomForest_D: 0.7410
LogisticRegression_D: 0.7106


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

model_classes = [
    #('KNN_D', KNeighborsClassifier()),
    #('DecisionTree_D', DecisionTreeClassifier()),
    #('RandomForest_D', RandomForestClassifier()),
    #('AdaBoost_D', AdaBoostClassifier()),
    ('LogisticRegression_D', LogisticRegression()),
    ('NaiveBayes_D', GaussianNB())
]
classical_sim(X_train[:12000], X_test, y_train[:12000], y_test, 112000)




LogisticRegression_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



NaiveBayes_D - training...

Final Results:
LogisticRegression_D: 0.6808
NaiveBayes_D: 0.6450


In [ ]:
model_classes = [
    ('NaiveBayes_D', GaussianNB())
]

classical_sim(X_train[:16000], X_test, y_train[:16000], y_test, 16000)
classical_sim(X_train, X_test, y_train, y_test, 20000)



NaiveBayes_D - training...

Final Results:
NaiveBayes_D: 0.6458

NaiveBayes_D - training...

Final Results:
NaiveBayes_D: 0.6432


In [ ]:
from sklearn import svm
from sklearn.svm import SVC

model_classes = [
    ('SVM_D', SVC()),
]
classical_sim(X_train[:12000], X_test, y_train[:12000], y_test, 1112000)



SVM_D - training...


In [ ]:
classical_sim(X_train[:16000], X_test, y_train[:16000], y_test, 16000)
classical_sim(X_train, X_test, y_train, y_test, 20000)

# GRID SEARCH

In [ ]:
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [ ]:
del X_train

In [ ]:
import time
import gc
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, roc_curve, auc
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


def classical_sim_grid(X_val, y_val, X_train, X_test, y_train, y_test, param="grid"):
    results = []

    model_classes = {
        'KNN_D': (KNeighborsClassifier(), {
            'n_neighbors': list(range(3, 26))
        }),
        'DecisionTree_D': (DecisionTreeClassifier(), {
            'max_depth': [5, 10, 15, 20, 25, 30, None]
        }),
        'RandomForest_D': (RandomForestClassifier(), {
            'n_estimators': [50, 100, 1000],
            'max_depth': [10, 20],
            'max_features': ['sqrt']
        }),
       
    }

    for name, (model, param_grid) in model_classes.items():
        print(f"\n{name} - Grid Search...")

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='accuracy',
            cv=3,
            n_jobs=-1,
            verbose=1
        )

        start_time = time.time()
        grid_search.fit(X_val, y_val)
        training_time = time.time() - start_time

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"Best params: {best_params}")

        start_time = time.time()
        model.set_params(**best_params)
        model.fit(X_train, y_train)
        end_time = time.time()
        training_time = end_time - start_time

        preds = model.predict(X_test)
        acc_preview = accuracy_score(y_test, preds)

        y_prob = model.predict(X_test)
        y_pred = y_prob
        y_true = y_test

        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred)

        filename_prefix = f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}"
        with open(f"{filename_prefix}_results.txt", "w") as f:
            f.write(f"Model: {name}\n")
            f.write(f"Best Parameters: {best_params}\n")
            f.write(f"Training time: {training_time:.2f} sec\n")
            f.write(f"Accuracy: {accuracy:.4f}\n\n")
            f.write("Classification Report:\n")
            f.write(report)

        conf_matrix = confusion_matrix(y_test, preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.title(f"{name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.savefig(f"{filename_prefix}_matrix.png")
        plt.close()

        try:
            if len(np.unique(y_test)) == 2:
                y_score = best_model.predict_proba(X_test)[:, 1]
                fpr, tpr, _ = roc_curve(y_test, y_score)
                roc_auc = auc(fpr, tpr)

                plt.figure(figsize=(8, 6))
                plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.2f})')
                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('ROC Curve')
                plt.legend(loc="lower right")
                plt.tight_layout()
                plt.savefig(f"{filename_prefix}_roc_curve.png")
                plt.close()
        except Exception as e:
            print(f"⚠️ ROC failed: {e}")

        results.append((name, best_params, accuracy))

        del grid_search, best_model, preds, conf_matrix
        gc.collect()

    with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_final_results.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Best Parameters", "Accuracy"])
        for name, best_params, acc in results:
            writer.writerow([name, str(best_params), f"{acc:.4f}"])

    print("\nFinal Results:")
    for name, best_params, acc in results:
        print(f"{name} with best parameters {best_params}: {acc:.4f}")


In [ ]:
classical_sim_grid(X_val, y_val, X_train_new, X_test, y_train_new, y_test, param = "_classical_grid_juneeee")



KNN_D - Grid Search...
Fitting 3 folds for each of 23 candidates, totalling 69 fits
Best params: {'n_neighbors': 4}

DecisionTree_D - Grid Search...
Fitting 3 folds for each of 7 candidates, totalling 21 fits
Best params: {'max_depth': 25}

RandomForest_D - Grid Search...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best params: {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 1000}

Final Results:
KNN_D with best parameters {'n_neighbors': 4}: 0.5725
DecisionTree_D with best parameters {'max_depth': 25}: 0.6314
RandomForest_D with best parameters {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 1000}: 0.7541
